## Query Translation

#### Этот этап отвечает за работу с запросом от пользователя

### Библиотеки и окружения

In [1]:
from necessity import *

Варинты разбивки документов, рассмотрим 3 самые лучшие по результатам с 1 этапа

In [ ]:
# # 1 вариант 
# splits = split_documents_html(chunk_size=1000, chunk_overlap=100)

# # 2 вариант 
# splits = split_documents_my_html()

# # 3 вариант 
# splits = split_documents_standart(chunk_size=1000, chunk_overlap=100)

## Multi-Query (Мультизапросы)

![structure.png](../Schemes/Multi-Query.png)

In [12]:
# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Query_Translation_Multi_Query_(html_split)"

In [2]:
# получаем смысловую разбивку всех документов
splits = split_documents_my_html()
# splits = split_documents_html(chunk_size=1000, chunk_overlap=100)
# splits = split_documents_standart(chunk_size=1000, chunk_overlap=100)

# индексируем
embeddings_model = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
db = Chroma.from_documents(documents=splits, embedding=embeddings_model)

# Устанавливаем ретриевер для поиска контекста
retriever = db.as_retriever(search_kwargs={'k': 1})

llm = GigaChat(model='GigaChat-Plus', verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [3]:
# Multi Query
prompt_perspectives = ChatPromptTemplate.from_template("""Вы - ИИ ассистент. 
            Ваша задача - сгенерировать три варианта исходного вопроса для поиска в векторной базе данных.
            Задайте эти вопросы с разных точек зрения. Представьте альтернативные вопросы, разделенные новой строкой.
            Исходный вопрос: {question}""")

generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

# список уникальных документов для контекста
def get_unique_docs(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

# Retrieve
# question = "Можно ли не расписываться на карте?"
retrieval_chain = generate_queries | retriever.map() | get_unique_docs
# docs = retrieval_chain.invoke({"question":question})

In [4]:
# RAG
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете.
        Question: {question} 
        Context: {context} 
        Answer:""")

final_rag_chain = (
    {"context": retrieval_chain, "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

# final_rag_chain.invoke("Можно ли не расписываться на карте?")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


'Не указано в контексте.'

### Загрузка Датасета

In [16]:
import pandas as pd
df = pd.read_csv('../dataset.csv')

df['answer'] = df['question'].apply(lambda x: final_rag_chain.invoke(x.strip()))
display(df)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,Не указано в контексте.
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...","Да, это может быть опасно. Мошенники могут исп..."
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно хранить пинкод в заметках в ..."
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...","Если вы потеряли карту, вам следует обратиться..."
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,Важно не расписываться на квитанциях/чеках до ...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста..."
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...","Нет, не рекомендуется использовать интернет-ка..."
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,Два способа обезопасить себя от съема данных к...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...",При бронировании гостиницы по карте вам потреб...


In [17]:
df.to_csv('../rag_answer/rag_step_query_translation/rag_multi_query.csv', index=False)

## RAG-Fusion 

![structure.png](../Schemes/Rag-Fusion.png)

In [29]:
# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Query_Translation_RAG_Fusion"

In [30]:
prompt_rag_fusion = ChatPromptTemplate.from_template("""
                        Вы - полезный помощник, который генерирует несколько поисковых запросов на основе одного входного запроса. \n
                        Сгенерируйте несколько поисковых запросов, связанных с: {question}.
                        Постарайтесь как будто разложить исохдный вопрос на более простые вопросы.\n 
                        Output (3 queries): \n""")

generate_queries_fusion = (
    prompt_rag_fusion |
    llm |
    StrOutputParser() |
    (lambda x: x.split("\n"))
)

# ранжирование документов (метод обратного ранжирования)
def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

chain_fusion = generate_queries_fusion | retriever.map() | reciprocal_rank_fusion

In [31]:
# RAG
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете.
        Question: {question} 
        Context: {context} 
        Answer:""")

final_rag_fusion_chain = (
    {"context": chain_fusion, "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

### Загрузка Датасета

In [32]:
import pandas as pd
df = pd.read_csv('../dataset.csv')

df['answer'] = df['question'].apply(lambda x: final_rag_fusion_chain.invoke(x.strip()))
display(df)

Failed to batch ingest runs: LangSmithError("Failed to post https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Read timed out. (read timeout=10.0)\n")


,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,Не указано в контексте.
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...","Да, передавать свою карту другому человеку для..."
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно хранить пинкод в заметках в ..."
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...","Если вы потеряли карту, вам следует немедленно..."
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,Подписывая квитанции или чеки до проверки прав...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста..."
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...","Нет, не рекомендуется использовать интернет-ка..."
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,Для защиты вашей карты от съема данных злоумыш...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...","Нет, при бронировании гостиницы по карте вам н..."


In [33]:
df.to_csv('../rag_answer/rag_step_query_translation/rag_fusion.csv', index=False)

## Step-back

![structure.png](../Schemes/Step_back.png)

In [29]:
# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Query_Translation_Step_back_(my_html_split)"

In [31]:
# Few Shot примеры
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "Нужно ли ИП указывать свой сайт для предприятия?",
        "output": "Что необходимо предоставить ИП?",
    },
    {
        "input": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?",
        "output": "Что будет при неиспользовании кредитной карты?",
    },
    {
        "input": "Дают ли кредит на автомобиль?",
        "output": "На что дают кредит?",
    },
    {
        "input": "Распростроняется ли страховое покрытие на смерть в алкогольном опьянении?",
        "output": "На что не распространяется страховое покрытие по риску 'Смерть'?",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# промт из статьи 
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Вы являетесь экспертом в области мировых знаний. Ваша задача - сделать шаг назад и перефразировать вопрос в более общий, на который проще ответить.
               Вот несколько примеров:""",
        ),
        few_shot_prompt,
        # исходный вопрос
        ("user", "{question}"),
    ]
)

generate_queries_step_back = prompt | llm | StrOutputParser()

In [33]:
prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
    Я собираюсь задать вам вопрос. Ваш ответ должен быть исчерпывающим и не противоречить следующему контексту, если он уместен.
    В противном случае, игнорируйте его, если он неуместен.

    # {normal_context}
    # {step_back_context}

    # Original Question: {question}
    # Answer:
""")

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# chain.invoke({"question": 'Можно ли не расписываться на карте?'})

### Загрузка Датасета

In [35]:
import pandas as pd
df = pd.read_csv('../dataset.csv')

df['answer'] = df['question'].apply(lambda x: chain.invoke({"question": x.strip()}))
display(df)

Failed to batch ingest runs: LangSmithError("Failed to post https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Read timed out. (read timeout=10.0)\n")


,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,"Нет, согласно пользовательскому соглашению бан..."
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...","Нет, это может быть опасно, так как ваша карта..."
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно. Хранение ПИН-кода в заметка..."
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...","Если Вы потеряли карту, Вам следует немедленно..."
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,Подпись на квитанции или чеке является подтвер...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста..."
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...","Нет, не рекомендуется использовать интернет-ка..."
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,Чтобы обезопасить себя от съема данных карты в...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...","Нет, при бронировании гостиницы по карте вам н..."


In [36]:
df.to_csv('../rag_answer/rag_step_query_translation/rag_step_back.csv', index=False)